In [ ]:
'''
Task 1
Fetch MQTTset from Kaggle
'''
import os
from kaggle.api.kaggle_api_extended import KaggleApi

# Authenticate using Kaggle API
api = KaggleApi()
api.authenticate()

# Download the dataset
dataset = 'cnrieiit/mqttset'
api.dataset_download_files(dataset, path='.', unzip=True)

In [1]:
'''
Task 1
Setup SparkSession and test connection to Postgres
'''
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit 


# Define dataset, spark constants
APPNAME = 'Roberts_Systool_Project'
MASTER = 'local[3]'
JDBC_JAR_PATH = f"/home/jrob/.cache/pypoetry/virtualenvs/systems-toolchains-C8y3MhrL-py3.10/lib/python3.10/site-packages/pyspark/jars/postgresql-42.7.4.jar"
JDBC_URL = "jdbc:postgresql://localhost:5432/postgres"
DB_PROPERTIES = {
    "user": "postgres",
    "password": "postgres_pw",
    "driver": "org.postgresql.Driver"
}

# Initialize Spark session with PostgreSQL JDBC driver
spark = SparkSession.builder \
    .appName(APPNAME) \
    .master(MASTER)\
    .config('spark.driver.host','127.0.0.1')\
    .config("spark.jars", JDBC_JAR_PATH) \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "2g") \
    .getOrCreate()

# df = spark.read.jdbc(url=JDBC_URL, table="mqtt", properties=DB_PROPERTIES)
# df.show()

your 131072x1 screen size is bogus. expect trouble
24/10/11 20:49:51 WARN Utils: Your hostname, Jonathan-Laptop resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/10/11 20:49:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/10/11 20:49:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [18]:
'''
Task 1
Ingest data with Spark and write to Postgres

Creates a dictionary for data files and associated data category,
then loops over both values using .items() method

In the loop, it dynamically changes the data category and write mode
based on where the code executes in the loop. First loop overwrites
to create table and schema if not already exists. Second loop appends
'''

DATA_FOLDER = "./data/FINAL_CSV" # won't change due to mqttset zip

data_files = {
    'train70.csv': 'train',
    'test30.csv': 'test'
}
overwrite = True
for file, set_type_value in data_files.items():
    file_path = os.path.join(DATA_FOLDER, file)
    
    # Read CSV file
    df = spark.read.csv(file_path, header=True, inferSchema=True)
    
    # Add 'set_type' column and dataset value of 'train' or 'test' based on place in dictionary loop
    df = df.withColumn('set_type', lit(set_type_value))
    
    df.repartition(6)
    # Show first 5 rows for verification
    df.show(5)
    
    # Overwrite on the first loop and append on the rest
    write_mode = 'overwrite' if overwrite else 'append'
    
    # Write data to Postgres
    df.write.jdbc(url=JDBC_URL, 
                  table="mqtt", 
                  mode=write_mode, 
                  properties=DB_PROPERTIES
                  batchsize=10000
                  )
    
    # Set overwrite to False after the first write
    if overwrite:
        overwrite = False

+----------+--------------+-------+-----------------+--------------------------+--------------------+---------------+----------------------+-------------------+----------------+---------------------+-------------------+------------------+---------------------+-------------+------------+-------------+-----------+--------+--------+----------+------------+--------------+--------------+--------+-----------+------------+---------------+--------+------------+----------------+--------------+------------------+----------+--------+
| tcp.flags|tcp.time_delta|tcp.len|mqtt.conack.flags|mqtt.conack.flags.reserved|mqtt.conack.flags.sp|mqtt.conack.val|mqtt.conflag.cleansess|mqtt.conflag.passwd|mqtt.conflag.qos|mqtt.conflag.reserved|mqtt.conflag.retain|mqtt.conflag.uname|mqtt.conflag.willflag|mqtt.conflags|mqtt.dupflag|mqtt.hdrflags|mqtt.kalive|mqtt.len|mqtt.msg|mqtt.msgid|mqtt.msgtype|mqtt.proto_len|mqtt.protoname|mqtt.qos|mqtt.retain|mqtt.sub.qos|mqtt.suback.qos|mqtt.ver|mqtt.willmsg|mqtt.willmsg_l

+----------+--------------+-------+-----------------+--------------------------+--------------------+---------------+----------------------+-------------------+----------------+---------------------+-------------------+------------------+---------------------+-------------+------------+-------------+-----------+--------+--------+----------+------------+--------------+--------------+--------+-----------+------------+---------------+--------+------------+----------------+--------------+------------------+----------+--------+
| tcp.flags|tcp.time_delta|tcp.len|mqtt.conack.flags|mqtt.conack.flags.reserved|mqtt.conack.flags.sp|mqtt.conack.val|mqtt.conflag.cleansess|mqtt.conflag.passwd|mqtt.conflag.qos|mqtt.conflag.reserved|mqtt.conflag.retain|mqtt.conflag.uname|mqtt.conflag.willflag|mqtt.conflags|mqtt.dupflag|mqtt.hdrflags|mqtt.kalive|mqtt.len|mqtt.msg|mqtt.msgid|mqtt.msgtype|mqtt.proto_len|mqtt.protoname|mqtt.qos|mqtt.retain|mqtt.sub.qos|mqtt.suback.qos|mqtt.ver|mqtt.willmsg|mqtt.willmsg_l

In [ ]:
spark.stop()

In [2]:
'''
Task 2
Ingest Data from Postgres into Spark for Data Analysis
'''
num_partitions = 6 # Adjust based on your environment
mqtt_df = spark.read.jdbc(url=JDBC_URL,
                          table="mqtt",
                          properties=DB_PROPERTIES,
                          numPartitions=num_partitions,
                          )


In [3]:
# Number of rows
num_rows = mqtt_df.count()

# Number of columns
num_cols = len(mqtt_df.columns)

print(f"Shape: {num_rows}, {num_cols}")

Shape: 12081189, 35


In [4]:
mqtt_df.printSchema()

root
 |-- tcp.flags: string (nullable = true)
 |-- tcp.time_delta: double (nullable = true)
 |-- tcp.len: integer (nullable = true)
 |-- mqtt.conack.flags: string (nullable = true)
 |-- mqtt.conack.flags.reserved: double (nullable = true)
 |-- mqtt.conack.flags.sp: double (nullable = true)
 |-- mqtt.conack.val: double (nullable = true)
 |-- mqtt.conflag.cleansess: double (nullable = true)
 |-- mqtt.conflag.passwd: double (nullable = true)
 |-- mqtt.conflag.qos: double (nullable = true)
 |-- mqtt.conflag.reserved: double (nullable = true)
 |-- mqtt.conflag.retain: double (nullable = true)
 |-- mqtt.conflag.uname: double (nullable = true)
 |-- mqtt.conflag.willflag: double (nullable = true)
 |-- mqtt.conflags: string (nullable = true)
 |-- mqtt.dupflag: double (nullable = true)
 |-- mqtt.hdrflags: string (nullable = true)
 |-- mqtt.kalive: double (nullable = true)
 |-- mqtt.len: double (nullable = true)
 |-- mqtt.msg: string (nullable = true)
 |-- mqtt.msgid: double (nullable = true)
 |-